In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'heart-failure-prediction:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1582403%2F2603715%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240821%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240821T124623Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D8de8f07d5298adc5696b57ca834f9f61f763e418c66c9e2c3c499d02d385347712c5658e34dadd2edd8046fe0dc0cbf271327f80871fa9318a683cc753427053b54d67950c7b3e71e09966deec1e89556b2f0760be9c91bd09b20564508f73474e6a3cfbb5260b3a3a52f551d24acd0acc23543c15a6f105254fc72c4d240694b2b5b0c25f0f3412f1563fa03a68e33a61549132ef2d57637d6f25d44a2d897cc6910e3229052cf5367afd69c4be8cbe72c8baf96681d94328dd464446f229e9a4e2cb9933f977ae8e0310538e3511816329ebe6894b8dda28b95ad37c978819b750c79cc4b5df076c042ac21d9cb5737b30c4ab82118111bb003cb90616d746'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


#### Kheirallah Samaha

#### 07-Dec-2022

### About the Dataset (according to the dataset's publisher in kaggle):

#### Context:

Cardiovascular diseases (CVDs) are the number 1 cause of death globally, taking an estimated 17.9 million lives each year, which accounts for 31% of all deaths worldwide. Four out of 5CVD deaths are due to heart attacks and strokes, and one-third of these deaths occur prematurely in people under 70 years of age. Heart failure is a common event caused by CVDs and this dataset contains 11 features that can be used to predict a possible heart disease.

People with cardiovascular disease or who are at high cardiovascular risk (due to the presence of one or more risk factors such as hypertension, diabetes, hyperlipidaemia or already established disease) need early detection and management wherein a machine learning model can be of great help.

#### Attribute Information:

Age: age of the patient [years]

Sex: sex of the patient [M: Male, F: Female]

ChestPainType: chest pain type [TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic]

RestingBP: resting blood pressure [mm Hg]

Cholesterol: serum cholesterol [mm/dl]

FastingBS: fasting blood sugar [1: if FastingBS > 120 mg/dl, 0: otherwise]

RestingECG: resting electrocardiogram results [Normal: Normal, ST: having ST-T wave abnormality (T wave inversions and/or ST elevation or depression of > 0.05 mV), LVH: showing probable or definite left ventricular hypertrophy by Estes' criteria]

MaxHR: maximum heart rate achieved [Numeric value between 60 and 202]

ExerciseAngina: exercise-induced angina [Y: Yes, N: No]

Oldpeak: oldpeak = ST [Numeric value measured in depression]

ST_Slope: the slope of the peak exercise ST segment [Up: upsloping, Flat: flat, Down: downsloping]

HeartDisease: output class [1: heart disease, 0: Normal]


## Load Packages

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("fivethirtyeight")

Check pandas version¶

In [ ]:
pd.__version__

## Import Data

In [ ]:
heart_dft = pd.read_csv("/kaggle/input/heart-failure-prediction/heart.csv")
heart_dft.head(), heart_dft.shape

Observations:

* We have a dataset with 12 variables and 918 rows
* We have different data types

## Get the dataset's informatio

In [ ]:
heart_dft.info()

Observations:

* Seven(7) numeric variables and five(5) variables type object
* No missing data in this Dataset
* Variables names in Pascal Case, I will keep it like that.
* will convert object variables to category data type

### Convert object vars and "FastingBS" var to Categorical data type

In [ ]:
heart_dft[
    [
        "ChestPainType",
        "Sex",
        "FastingBS",
        "RestingECG",
        "ExerciseAngina",
        "ST_Slope",
        "HeartDisease",
    ]
] = heart_dft[
    [
        "ChestPainType",
        "Sex",
        "FastingBS",
        "RestingECG",
        "ExerciseAngina",
        "ST_Slope",
        "HeartDisease",
    ]
].astype(
    "category"
)

### Replace "M" with Male and "F" with Female and HeartDisease to Yes and No.

In [ ]:
heart_dft["Sex"] = heart_dft["Sex"].map({"M": "Male", "F": "Female"})
heart_dft["Sex"]
heart_dft["HeartDisease"] = heart_dft["HeartDisease"].map({0: "No", 1: "Yes"})
heart_dft.info()

lets use describe() for summary statistics!

In [ ]:
heart_dft.describe().round()

### Counts of each Calegorical Vars ("ChestPainType", "Sex", "RestingECG", "ExerciseAngina", "ST_Slope")

#### Sex Var

In [ ]:
sex_counts = heart_dft["Sex"].value_counts(normalize=True).round(2) * 100
sex_counts = sex_counts.reset_index().rename(columns={"Sex": "Pct", "index": "Sex"})
sex_counts

In [ ]:
plt.figure(figsize=(7, 5))
sns.set_context("paper")

sex_color = dict({"Male": "#2986cc", "Female": "#c90076"})

ax1 = sns.barplot(
    data=sex_counts,
    x="Sex",
    #errorbar=None,
    y="Pct",
    palette=sex_color,
    linewidth=0.5,
    edgecolor="black",
    alpha=0.7,
)

values1 = ax1.containers[0].datavalues
labels = ["{:g}%".format(val) for val in values1]
ax1.bar_label(ax1.containers[0], labels=labels)

ax1.set_ylabel("Percent")
ax1.set_xlabel("")
ax1.set_title(
    "Almost 80% percent of the gender category are Males, ~21% Females", fontsize=10
)


plt.show()

Observations:

* Imbalanced data in terms of the Sex variable

#### ChestPainType Vs Sex

In [ ]:
ChestPainType_vs_Sex = (
    heart_dft[["ChestPainType", "Sex"]]
    .value_counts(normalize=True)
    .reset_index(name="Pct")
    .sort_values(by="ChestPainType")
)
ChestPainType_vs_Sex["Pct"] = ChestPainType_vs_Sex["Pct"].round(2) * 100
ChestPainType_vs_Sex

### RestingECG

In [ ]:
RestingECG_vs_Sex = (
    heart_dft[["RestingECG", "Sex"]]
    .value_counts(normalize=True)
    .reset_index(name="Pct")
    .sort_values(by="RestingECG")
)
RestingECG_vs_Sex["Pct"] = RestingECG_vs_Sex["Pct"].round(2) * 100
RestingECG_vs_Sex.sort_values(by="Pct", ascending=False)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

palette4 = {"ASY": "#1b85b8", "ATA": "#5a5255", "NAP": "#559e83", "TA": "#ae5a41"}

palette5 = {"LVH": "#2dc937", "Normal": "#e7b416", "ST": "#cc3232"}

sns.barplot(
    data=ChestPainType_vs_Sex,
    x="Sex",
    hue="ChestPainType",
    #errorbar=None,
    y="Pct",
    palette=palette4,
    linewidth=0.5,
    edgecolor="black",
    alpha=0.8,
    ax=ax[0],
)

for ax1 in [ax[0]]:
    for container in ax1.containers:
        values2 = container.datavalues
        labels = ["{:g}%".format(val) for val in values2]
        ax1.bar_label(container, labels=labels)

ax[0].set_ylabel("Percent")
ax[0].set_xlabel("")
ax[0].set_title(
    "Regardless of the proportion of Males and Females,\n Men have high ASY compared with Women, and the pattern is different.",
    fontsize=10,
)


sns.barplot(
    data=RestingECG_vs_Sex,
    x="Sex",
    hue="RestingECG",
    #errorbar=None,
    y="Pct",
    palette=palette5,
    linewidth=0.5,
    edgecolor="black",
    alpha=0.8,
    ax=ax[1],
)

for ax2 in [ax[1]]:
    for container in ax2.containers:
        values3 = container.datavalues
        labels = ["{:g}%".format(val) for val in values3]
        ax2.bar_label(container, labels=labels)

ax[1].set_ylabel("")
ax[1].set_xlabel("")
ax[1].set_title("Men and Women have somehow same pattern of RestingECG", fontsize=10)


plt.tight_layout()
plt.show()

Observations:

* ChestPainType: chest pain type [TA: Typical Angina, ATA: Atypical Angina, NAP: Non-Anginal Pain, ASY: Asymptomatic]

The majority 46% of the Male participants are in ASY category

* RestingBP: resting blood pressure taking a rest for min five(5) minutes before blood pressure measurement

The majority of the participants are in the Normal category

Note: The percentage is calculated using all participants, NOT within each group of Sex

### ExerciseAngina and ST_Slope

In [ ]:
ExerciseAngina_vs_Sex = (
    heart_dft[["ExerciseAngina", "Sex"]]
    .value_counts(normalize=True)
    .reset_index(name="Pct")
    .sort_values(by="ExerciseAngina")
)
ExerciseAngina_vs_Sex["Pct"] = ExerciseAngina_vs_Sex["Pct"].round(2) * 100


ST_Slope_vs_Sex = (
    heart_dft[["ST_Slope", "Sex"]]
    .value_counts(normalize=True)
    .reset_index(name="Pct")
    .sort_values(by="ST_Slope")
)
ST_Slope_vs_Sex["Pct"] = ST_Slope_vs_Sex["Pct"].round(2) * 100

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(12, 5), sharey=True)

palette6 = {
    "Y": "#000000",
    "N": "#009900",
}

palette7 = {"Down": "#b2d8d8", "Flat": "#66b2b2", "Up": "#004c4c"}

sns.barplot(
    data=ExerciseAngina_vs_Sex,
    x="Sex",
    hue="ExerciseAngina",
    #errorbar=None,
    y="Pct",
    palette=palette6,
    linewidth=0.5,
    edgecolor="black",
    alpha=0.8,
    ax=ax[0],
)

for ax3 in [ax[0]]:
    for container in ax3.containers:
        values2 = container.datavalues
        labels = ["{:g}%".format(val) for val in values2]
        ax3.bar_label(container, labels=labels)

ax[0].set_ylabel("Percent")
ax[0].set_xlabel("")
ax[0].set_title(
    "Almost a similar pattern between Men and Women. (ExerciseAngina)", fontsize=10
)


sns.barplot(
    data=ST_Slope_vs_Sex,
    x="Sex",
    hue="ST_Slope",
    #errorbar=None,
    y="Pct",
    palette=palette7,
    linewidth=0.5,
    edgecolor="black",
    alpha=0.8,
    ax=ax[1],
)

for ax4 in [ax[1]]:
    for container in ax4.containers:
        values3 = container.datavalues
        labels = ["{:g}%".format(val) for val in values3]
        ax4.bar_label(container, labels=labels)

ax[1].set_ylabel("")
ax[1].set_xlabel("")
ax[1].set_title(
    "A different pattern between Men and Women (ExerciseAngina)", fontsize=10
)


plt.tight_layout()
plt.show()

Note:

* ExerciseAngina: exercise-induced angina [Y: Yes, N: No]
Angina is a type of chest pain caused by reduced blood flow to the heart

* ST_Slope: the slope of the peak exercise ST segment [Up: upsloping, Flat: flat, Down: downsloping]
Mainly Flat in (Males) and relatively Up in (Females)

### Numeric Vars ("Age", "Cholesterol", "MaxHR", "Oldpeak")

In [ ]:
filtheart_dft = heart_dft["Cholesterol"] > 0
heart_dft_chol_n0 = heart_dft[filtheart_dft]

sns.JointGrid(
    data=heart_dft, x="Age", y="Cholesterol", hue="Sex", palette=sex_color
).plot(sns.scatterplot, sns.histplot)
plt.show()

Observations:

* It is somehow the same pattern for Male and Female relative to their counts!
* We have many of 0 values in the Cholesterol var!!

* Let's use lmplot to see the regression line of each group.
* I think we have to remove the 0 values in the Cholesterol var and check again!

In [ ]:
sns.lmplot(
    data=heart_dft_chol_n0,
    x="Age",
    y="Cholesterol",
    hue="Sex",
    palette=sex_color,
).set(ylabel="Cholesterol", xlabel="Age", title="")
plt.show()

* We may need to do some t-test to check whether the true means difference is equal to 0 or Not. we can scipy package to do that, but for now, let's move on...

### Find the mean Cholesterol by Sex

In [ ]:
Chol_mean_f = (
    heart_dft_chol_n0[["Sex", "Cholesterol"]]
    .groupby(["Sex"])
    .mean("Cholesterol")
    .loc["Female", "Cholesterol"]
).round()
Chol_mean_m = (
    heart_dft_chol_n0[["Sex", "Cholesterol"]]
    .groupby(["Sex"])
    .mean("Cholesterol")
    .loc["Male", "Cholesterol"]
).round()

print("for Female =", Chol_mean_f, "for Male =", Chol_mean_m)

* Somehow Same mean

let's try to plot the result!

### Cholesterol var, density distribution, Male VS Female

In [ ]:
plt.figure(figsize=(10, 5))
sns.set_context("paper")

kdeplt = sns.kdeplot(
    data=heart_dft_chol_n0,
    x="Cholesterol",
    hue="Sex",
    palette=sex_color,
    alpha=0.7,
    lw=2,
)
kdeplt.set_title("Cholesterol values distribution\n Male VS Female", fontsize=12)
kdeplt.set_xlabel("Cholesterol", fontsize=12)


plt.axvline(x=Chol_mean_f, color="#c90076", ls="--", lw=1.3)
plt.axvline(x=Chol_mean_m, color="#2986cc", ls="--", lw=1.3)
plt.text(108, 0.00612, "Mean Cholesterol / Male", fontsize=10, color="#2986cc")
plt.text(260, 0.006, "Mean Cholesterol / Female", fontsize=10, color="#c90076")


plt.show()

* Indeed, they are similar, regardless of the sample size of females.
* Again, to make a sufficient result, we have to do t-test or linear regression.

In [ ]:
heart_dft.info()

### Target Variable - HeartDisease

#### Counts

In [ ]:
HeartDisease_counts = (
    heart_dft[["HeartDisease"]]
    .value_counts(normalize=True)
    .reset_index(name="Pct")
    .sort_values(by="HeartDisease")
)
HeartDisease_counts["Pct"] = HeartDisease_counts["Pct"].round(2) * 100
HeartDisease_counts

* Somehow balanced

In [ ]:
HeartDisease_colors = {"Yes": "#d62d20", "No": "#008744"}
hplt = sns.barplot(
    data=HeartDisease_counts,
    x="Pct",
    #errorbar=None,
    y="HeartDisease",
    palette=HeartDisease_colors,
    linewidth=0.5,
    edgecolor="black",
    alpha=0.7,
)

hplt.set_ylabel("HeartDisease", rotation=45)
hplt.set_xlabel("Percent")
hplt.set_title("Acceptably balanced target variable ", fontsize=10)
plt.show()

### Pairplot

#### Removing zeros from Oldpeak and Cholesterol vars

In [ ]:
filtered_for_pairplot = heart_dft.loc[
    (heart_dft["Oldpeak"] > 0) & (heart_dft["Cholesterol"] > 0)
]

#### Creats a sample of 150 to speed up the ploting process

In [ ]:
pairplot_df = (
    filtered_for_pairplot.groupby(["Sex"])  # Stratify by "Sex"
    .apply(lambda x: x.sample(75))  # 75 for each Sex
    .reset_index(drop=True)  # int64 index type, back to default!
)

In [ ]:
sns.pairplot(
    pairplot_df,
    hue="HeartDisease",
    kind="scatter",
    diag_kind="kde",
    palette=HeartDisease_colors,
    height=1.5,
    aspect=1,
    plot_kws=dict(s=5),
)
plt.show()

Observations:

* In general, there is no big difference in the diagonal plot, Oldpeak and cholesterol however, have some extreme data points

In [ ]:
HeartDisease_by_Sex = (
    heart_dft.groupby(["Sex"])["HeartDisease"]
    .value_counts(normalize=True)
    .reset_index(name="Pct")
).rename(columns={"level_1": "HeartDisease"})

HeartDisease_by_Sex["Pct"] = HeartDisease_by_Sex["Pct"].round(2) * 100

HeartDisease_by_Sex

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 5))

hplt = sns.barplot(
    data=HeartDisease_by_Sex,
    x="Sex",
    #errorbar=None,
    y="Pct",
    hue="HeartDisease",
    linewidth=0.5,
    edgecolor="black",
    palette=HeartDisease_colors,
    alpha=0.6,
    ax=ax,
)

for axhs in [ax]:
    for container in axhs.containers:
        values_hs = container.datavalues
        labels = ["{:g}%".format(val) for val in values_hs]
        axhs.bar_label(container, labels=labels)

ax.set_ylabel("")
ax.set_xlabel("")
ax.set_title("", fontsize=10)


plt.tight_layout()
plt.show()

plt.show()

Note:

* According to this dataset, 26% of females have developed Heart Disease, and 74% Have not. and

* 63% of Males have developed Heart Disease, and 37% have not.

#### Lets check the maximum heart rate by age and Sex using FacetGrid

In [ ]:
heart_df_fg = sns.FacetGrid(
    data=heart_dft_chol_n0,
    col="Sex",
    hue="Sex",
    row="HeartDisease",
    height=4,
    aspect=1.3,
    palette=sex_color,
    col_order=["Male", "Female"],
)
heart_df_fg.map_dataframe(sns.regplot, "Age", "MaxHR")
plt.show()

Note:

* We can see that the correlation between Age and MaxHR in Males is negative in both heart disease categories

* Females have got positive correlation between Age and MaxHR in the heart disease category Yes, and a negative correlation in the heart disease category "No"

* It is worth noting that the range of Age range of females in category Yes is from ~ 30 to ~65, and it is from ~30 to ~ 75 in category No

* The range Age of Males in the category Yes is from ~ 30 to ~70, however it is from ~30 to ~ 77 in the category No

#### Lets find the mean of Age, Cholesterol, MaxHR and RestingBP, by sex and HeartDisease vars!

In [ ]:
heart_dft_chol_n0.groupby(["Sex", "HeartDisease"]).agg(
    Age_mean=pd.NamedAgg("Age", "mean"),
    Cholesterol_mean=pd.NamedAgg("Cholesterol", "mean"),
    MaxHR_mean=pd.NamedAgg("MaxHR", "mean"),
    RestingBP_mean=pd.NamedAgg("RestingBP", "mean"),
).reset_index().round(2)

Observations:

Females:

* The Cholesterol_mean value in females who developed heart disease is higher (279) than in those who did not (249)

* The RestingBP_mean value in females who developed heart disease is also higher (143) than in those who did not (128.7)

* MaxHR_mean in females who did develop heart disease is LOWER (139) than in those who did not (149)

Males:

* The Cholesterol_mean value in Males who developed heart disease is higher (247) than in those who did not (232)

* The RestingBP_mean value in Males who developed heart disease is slightly higher (135) than in those who did not (130.9)

* MaxHR_mean in Males who did develop heart disease is LOWER (129) than in those who did not (148.9)

Thank you for reading, and your comments are greatly appreciated.